In [58]:
NEO4J_URI="neo4j+s://db5fa286.databases.neo4j.io"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD="PjOlu1HKKmvPAaQDCwIusaQcqSNlPhkJ_jZQU1mrf0Y"


In [59]:
import os
os.environ["NEO4J_URI"]=NEO4J_URI
os.environ["NEO4J_USERNAME"]=NEO4J_USERNAME
os.environ["NEO4J_PASSWORD"]=NEO4J_PASSWORD


In [60]:
from langchain_community.graphs import Neo4jGraph
graph=Neo4jGraph(url=NEO4J_URI,username=NEO4J_USERNAME,password=NEO4J_PASSWORD)
graph

In [61]:
#DATASET MOVIES
movie_query="""
LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/movies/movies_small.csv' as row
MERGE (m:Movie{id:row.movieId})
SET m.released=date(row.released),
    m.title=row.title,
    m.imdbRating=toFloat(row.imdbRating)
FOREACH(director in split(row.director,'|')|
    MERGE (p:Person{name:trim(director)})
    MERGE (p)-[:DIRECTED]->(m))
FOREACH(actor in split(row.actor,'|')|
    MERGE (p:Person{name:trim(actor)})
    MERGE (p)-[:ACTED_IN]->(m))
FOREACH(genre in split(row.genres,'|')|
    MERGE (g:Genre{name:trim(genre)})
    MERGE (m)-[:IN_GENRE]->(g))
"""

In [62]:
movie_query

"\nLOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/movies/movies_small.csv' as row\nMERGE (m:Movie{id:row.movieId})\nSET m.released=date(row.released),\n    m.title=row.title,\n    m.imdbRating=toFloat(row.imdbRating)\nFOREACH(director in split(row.director,'|')|\n    MERGE (p:Person{name:trim(director)})\n    MERGE (p)-[:DIRECTED]->(m))\nFOREACH(actor in split(row.actor,'|')|\n    MERGE (p:Person{name:trim(actor)})\n    MERGE (p)-[:ACTED_IN]->(m))\nFOREACH(genre in split(row.genres,'|')|\n    MERGE (g:Genre{name:trim(genre)})\n    MERGE (m)-[:IN_GENRE]->(g))\n"

In [63]:
graph.query(movie_query)

[]

In [64]:
graph.refresh_schema

<bound method Neo4jGraph.refresh_schema of <langchain_community.graphs.neo4j_graph.Neo4jGraph object at 0x000001E1A3C7E810>>

In [65]:
import os
from dotenv import load_dotenv
load_dotenv()
groq_api_key=os.getenv("GROQ_API_KEY")

In [66]:
from langchain_groq import ChatGroq
llm=ChatGroq(groq_api_key=groq_api_key,model_name="gemma2-9b-it")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001E1A43BBF50>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001E1A43BA720>, model_name='gemma2-9b-it', groq_api_key=SecretStr('**********'))

In [17]:
!pip install langchain

In [67]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# Cypher generation chain
cypher_prompt = PromptTemplate(
    input_variables=["question"],
    template="Convert the following question to a Cypher query: {question}"
)
cypher_generation_chain = LLMChain(llm=llm, prompt=cypher_prompt)

qa_prompt = PromptTemplate(
    input_variables=["result", "query"],
    template="Here is the Cypher query that was executed: {query}\n\nAnswer based on the result: {result}"
)
qa_chain = LLMChain(llm=llm, prompt=qa_prompt)

# Step 5: Define the Graph Schema (this is a simplified example)
graph_schema = """
type Movie {
  id: ID!
  title: String!
  released: Int
  imdbRating: Float
}

type Person {
  name: String!
}

type Genre {
  name: String!
}
"""

# Step 6: Initialize the GraphCypherQAChain with all required parameters
chain = GraphCypherQAChain(
    graph=graph,
    cypher_generation_chain=cypher_generation_chain,
    qa_chain=qa_chain,
    graph_schema=graph_schema,
    verbose=True
)


In [70]:

query = "Which movies were directed by tom?"
result = chain.run(query)
print(result)






> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (m:Movie)-[:DIRECTED_BY]->(t:Director {name: "tom"})
RETURN m

Full Context:
[]


ValueError: Missing some input keys: {'result', 'query'}